In [3]:
import json
from backend.db.products import product_db, Product, VehicleType, Powertrain, Color
from pprint import pprint 
import traceback

# Problem with normalization of Mini MPV
def normalize(value):
    return "Mini MPV" if value == "Mini Mpv" else value

file_name = "test.json"
with open(file_name, 'r') as file:
    data = json.load(file)
    product_list = []
    for listing in data:
        try:
            product = Product(
                name=listing["heading"],
                vehicle_type=VehicleType(normalize(listing["build"]["body_type"])),
                release_year=listing["build"]["year"],
                price_in_usd=listing.get("price") or listing.get("ref_price") or listing.get("msrp") or float('nan') ,
                is_used=listing["inventory_type"]=="used",
                powertrain_type=Powertrain["EV" if listing["build"].get("powertrain_type") == "BEV" else "ICE" if listing["build"].get("powertrain_type") == "Combustion" else "Hybrid"],
                num_seats=listing["build"]["std_seating"],
                color=listing.get("base_ext_color") or "n/a",
                specs=listing["build"],
                image_url=listing["media"]["photo_links"] if listing.get("media") else [])
            
        except Exception as e:
            traceback.print_exc()
            print("Error in parsing the following listing:")
            pprint(listing)
            exit(1)
        
        product_list.append(product)
    
    await product_db.reset()
    await product_db.add(product_list)

In [4]:
x = await product_db.size()
print(x)

2874
